# 2.8 部署模型到生产环境中
## 🚄 前言
在经过模型微调和评测后，我们的开发工作已经接近完成。本节课程将介绍最后一部分的工作：将你的模型部署在计算资源上，以调用微调后的模型，或提升模型的并发性能。

## 🍁 课程目标
学完本节课程后，你将能够：
- 理解模型部署的目的与必要性，确保模型可以在不同环境中正常运行。
- 掌握本地和云端模型的部署方法，包括使用 vLLM 和部署服务方案的选择。

## 📖 课程目录

- 1.&nbsp; 模型本地部署
    - 1.1&nbsp; 使用 vLLM 部署模型
        - 1.1.1&nbsp; 准备环境
        - 1.1.2&nbsp; 启动vLLM服务
    - 1.2 测试服务
- 2.&nbsp; 模型云端部署
    - 2.1&nbsp; ECS云服务器部署
    - 2.2&nbsp; FC 函数计算
    - 2.3&nbsp; PAI-EAS 平台
    - 2.4&nbsp; 百炼平台部署
- 3.&nbsp;模型部署的更多场景

## 🚀 1. 模型本地部署
接下来，我们将了解如何在本地部署微调后的模型。

### 1.1 使用 vLLM 部署模型
vLLM 是一种专门用于加速大模型推理的部署工具。它支持多模型并行处理，并提供了丰富的 API 接口，让你在本地轻松启动模型。它有以下特点：
   - 高效并发管理：vLLM 能同时处理大量请求，非常适合高并发的任务。
   - 内存优化：通过碎片管理和内存分配，显著降低内存占用。
   - 改进缓存策略：引入创新缓存技术，减少重复计算，加快响应。
   - 灵活部署：支持多种语言模型，包括基础模型和微调模型，能够满足多样化需求。

现在，你将学会如何用 vLLM 部署一个 OpenAI 兼容的模型服务。这种服务可以让你的模型在本地环境中快速上线，为团队提供更高效的内容生产。

#### 1.1.1 准备环境
1、参考2.7章节中配置 ModelScope Notebook 的 GPU 实例

2、安装vLLM依赖项


In [1]:
! pip install vllm==0.6.2

#### 1.1.2 启动vLLM服务
vLLM 支持启动与 OpenAI API 兼容（即按照OpenAI API标准返回推理结果）的本地服务器。

按下图操作，打开一个新的 Terminal 终端窗口。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01jNwXHg23YLDPkJZ24_!!6000000007267-0-tps-1790-1604.jpg" width="400">

确保 Terminal 终端显示的路径名与本章节所处路径（/mnt/workspace/aliyun_acp_learning/大模型ACP认证教程/p2_构造大模型问答系统）一致，然后将以下命令中的模型路径替换为你的模型文件路径（本课程使用2_7章节中融合后模型的路径），即可启动本地部署。

> 如果端口被占用，请尝试使用其他端口号。

<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
<table width="90%">
<tbody>
<tr>
<td>

```bash
vllm serve "output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxxxx/checkpoint-xxx-merged" --load-format "safetensors" --port 8000 --dtype=half
```
</td>
</tr>
</tbody>
</table>

当Terminal中输出“INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)”，表示微调后的模型服务在本地启动成功且正常运行。

> 建议保持Terminal为运行状态，避免服务停止。

您可以返回此页面继续下一步操作。

### 1.2 测试服务
服务启动后，可以通过 curl 或 Postman 发送请求，验证模型响应是否正确。

将以下curl代码中model参数的路径修改为你的模型文件路径然后运行，测试模型的回答效果：

In [ ]:
%%bash
curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "/output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxxxx/checkpoint-xxx-merged",
        "messages": [
            {"role": "system", "content": "你是一个帮助用户的助手。"},
            {"role": "user", "content": "请告诉我2008年北京奥运会，中国队获得了多少枚金牌？"}
        ]
    }'

若收到合理回答，则表示服务可以正常使用。接下来，可以将此服务集成到机器人应用中，让模型即时生产内容。

## ☁2. 云上部署模型
随着业务扩展和员工需求增长，仅依靠本地部署无法满足公司的高并发和快速响应需求。

将模型迁移到云端并选择合适的解决方案，能让答疑机器人更有效地支持员工和客户。

本节将介绍几种常见的阿里云部署方案，帮助不同规模和需求的企业找到理想的选择：

   - ECS 云服务器：适合有运维团队、需要自主管控和长时间稳定运行的企业。灵活的资源配置让企业能够应对业务峰谷的变化，尤其适合对自定义有高需求的用户。
   - 函数计算 FC：为中小型企业提供便捷的按需扩展，适合灵活的短期或定时任务。Serverless 模式下，FC 支持按调用计费，能有效降低运维压力。
   - PAI-EAS 平台：针对实时推理和 AI 模型服务需求较高的企业，PAI-EAS 提供自动扩展和版本管理，适合对并发和服务持续性有严格要求的业务。
   - 百炼平台：适合需要闭源或托管模型服务的企业。百炼的独占实例管理支持资源动态调整和闭源模型的高并发访问，便于没有专业算法团队的企业高效部署。
   
通过了解并选择合适的云部署方案，你的答疑机器人可以轻松应对咨询高峰，保障员工的使用体验。

### 2.1 ECS 云服务器部署
ECS 云服务器适用于长期运行和高峰时段的灵活扩展。借助 Docker 容器管理资源，企业可以在不同负载下自主调整性能。

配置建议：
   - 资源选择：按需求选择 GPU 类型，如 Tesla T4、V100、A100 等。
   - 安全与成本管理：通过安全组规则保护网络，并在低峰期减少 GPU 实例数量。

部署步骤：
   - 配置 Docker 环境。
   - 安装 vLLM，启动模型推理服务。
   - 配置监控，确保稳定运行。

如果您的组织具备基于GPU 服务器部署模型的丰富经验，不想改变现有的工作模式，您可以直接使用阿里云ECS云服务器部署。详细步骤请参考：
- [使用vLLM容器镜像快速构建大预言模型在GPU上的推理环境](https://help.aliyun.com/zh/egs/use-cases/use-a-vllm-container-image-to-run-inference-tasks-on-a-gpu-accelerated-instance)

### 2.2 FC 函数计算
FC 函数计算适合初创公司和定期任务，提供按需扩展的 Serverless 和非 Serverless GPU 模式，降低长期闲置资源成本。

通过 函数计算（FC） 的 Serverless GPU 模式，公司能够按需扩展，不必支付长期闲置资源费用的同时也可以选择 非Serverless部署 以在成本和性能之间找到适合的平衡。然而，冷启动时间较长，因此更适合定期查询类任务，不适合高频实时问答。

阿里云 FC 提供事件驱动的部署方案，适合按需调用的灵活场景。FC 支持根据请求量自动扩展或缩容，适合成本敏感的短时高并发任务。

配置建议：
   - 实例选择：适合中小型推理任务，灵活选择配置。
   - 按需扩展与成本优化：按需扩展，低峰时释放资源，优化成本。
 
部署步骤：
   - 创建函数应用，选择运行环境。
   - 上传 Docker 镜像并配置。
   - 指定函数入口及模型路径。

FC 适合轻量化、低成本、高扩展性的需求。详细参考：
- [实时推理场景基本信息](https://help.aliyun.com/zh/functioncompute/fc-2-0/use-cases/real-time-inference-scenarios)

- [准实时推理场景的容器支持](https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/quasi-real-time-inference-scenarios)

- [实时推理场景的部署方式](https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/real-time-inference-scenarios-1)

### 2.3 PAI-EAS 平台

PAI-EAS 平台适合需要实时推理的大型企业。PAI-EAS 平台为模型提供智能扩展支持，能够自动化应对高并发请求，并通过模型版本管理与灰度发布功能，保障模型在更新过程中的平稳过渡。

同时，PAI-EAS 平台在开发环境和分布式训练上具备全面支持，便于企业在微调、预训练和上线过程中的各项管理。这使得 PAI-EAS 成为大型企业优化部署的高效选择。

部署建议：
   - 实例选择：推荐 ml.g7i.c16m60.1-gpu30 支持高并发。
   - 安全与监控：启用 CloudMonitor 实时监控，调整安全组规则保护数据。
 
部署步骤：
   - 创建自定义部署，设定模型路径。
   - 启用 vLLM 加速引擎，提升推理效率。
   - 测试和验证模型响应。
 
PAI-EAS 适合需要自动化管理和快速响应的场景。详细步骤请参考：
- [LLM大语言模型部署EAS服务](https://help.aliyun.com/zh/pai/user-guide/deploy-an-llm/) 

- [5分钟使用EAS一键部署LLM大语言模型应用常见问题及解决方法](https://help.aliyun.com/zh/pai/use-cases/deploy-llm-in-eas)

- [LLM智能路由：提升LLM推理系统整体效率部署服务](https://help.aliyun.com/zh/pai/user-guide/use-llm-intelligent-router-to-improve-inference-efficiency)

### 2.4 百炼平台部署
百炼平台智能管理闭源模型，适合需要高并发处理且无复杂运维的企业。

当业务需要调用闭源或托管模型时，百炼平台是理想选择，支持托管如通义千问等模型，为高并发场景提供独占实例管理。

百炼平台核心功能：
   - 创建服务：通过 API 创建模型任务，设定独占实例数量。
   - 自动扩缩容：根据需求动态调整实例，支持高峰负载。
 
部署步骤：

百炼的线上模型部署流程可有效简化管理工作，同时确保模型并发处理能力的灵活扩展。详细方法可参考以下文档：

- [大模型服务平台百炼的模型部署](https://help.aliyun.com/zh/model-studio/user-guide/model-deployment)

## 📊 3. 模型部署的更多场景
在实际应用中，模型部署通常用于满足以下三种场景:

- 应对高并发：当机器人在高并发环境中运行时，稳定性和响应速度尤为关键。模型的部署可以灵活调控资源，在高峰期自动扩展并保障流畅性。

- 微调后的特定需求：许多企业对机器人有特定的任务需求，需要对模型进行微调。通过部署微调后的模型至本地或云端，能够保证机器人在准确性和响应速度上的进一步提升，并支持持续迭代。

- 离线和本地环境：在某些工厂或高安全性组织中，网络连接可能受限甚至中断，机器人在此情况下需要本地资源支持，以保证持续运作。这种部署确保在无网络时，仍可稳定运行。

想象一下，答疑机器人主要使用云端部署模型高效处理复杂的请求和高并发需求。

另一方面，在特殊需求或网络限制的情况下，本地化部署的机器人则能够作为替代方案以满足严格的应急需求，一些机构或组织选择将模型本地部署，以便在特定网络条件或环境下独立运行并应对突发事件。

通过合理的部署策略，我们不仅能够提升机器人在高并发时的处理能力，还能确保数据安全、节省资源成本。

了解何时以及如何进行模型部署，将为答疑机器人提供更加灵活、稳定的支持。

## ✅本节小结

本节课程详细介绍了模型部署的目的和基本方法，你学习了：
- 模型部署的意义，以及何时需要部署模型。
- 模型本地部署的方法，包括微调模型和开源模型，以提供模型推理服务。
- 模型云上部署的方法，以提高模型的并发处理能力。

通过本节的学习，你已经掌握了模型部署的基本方法，为构建高性能、可扩展的大模型应用打下了坚实的基础。

接下来，你将学习一些借助大模型辅助内容生产的案例，如翻译、音视频转换、PPT制作等。希望这些案例能启发你开发更多效率工具。

>⚠️ **注意**：完成本节学习后，请及时停止当前的 PAI-DSW GPU 实例，以避免产生额外费用。
> 
>  进入第 3 部分学习前，建议继续使用2_7章节前所创建的 PAI-DSW CPU 实例。